In [3]:
import numpy as np
import cv2 as cv

def smooth_trajectory(points, window=5):
    """ Усредняет координаты траектории по скользящему окну """
    if len(points) < window:
        return points  # Если точек мало, не сглаживаем
    smoothed = np.convolve(points, np.ones(window)/window, mode='valid')
    return np.concatenate((points[:window-1], smoothed))  # Сохраняем начало

def remove_outliers(points, threshold=10):
    """ Убирает точки, которые резко отличаются от среднего движения """
    diffs = np.linalg.norm(np.diff(points, axis=0), axis=1)
    mask = diffs < threshold  # Оставляем только нормальные сдвиги
    return points[np.insert(mask, 0, True, axis=0)]  # Вставляем первую точку обратно



def process_video_segment(cap, start_frame, duration, fps):
    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    
    feature_params = dict(maxCorners=50, qualityLevel=0.6, minDistance=10, blockSize=45)
    lk_params = dict(winSize=(200, 200), maxLevel=3, 
                     criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 60, 0.01))
    color = np.random.randint(0, 255, (100, 3))
    
    ret, old_frame = cap.read()
    if not ret:
        return
    old_frame = old_frame[:, :450]
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    mask = np.zeros_like(old_gray)
    mask[:960, :-30] = 255  # Верхняя половина белая
    p0 = cv.goodFeaturesToTrack(old_gray, mask=mask, **feature_params)

    if p0 is None or len(p0) == 0:
        print(f"No features detected in the frame {start_frame}. Skipping this segment.")
        return
    
    mask = np.zeros_like(old_frame)
    
    frame_count = 0
    motion_vectors = []
    
    while frame_count < duration * fps:
        ret, frame = cap.read()
        if not ret:
            break
        frame = frame[:, :450]
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            #good_new[:, 0] = smooth_trajectory(good_new[:, 0])  # Сглаживание X
            #good_new[:, 1] = smooth_trajectory(good_new[:, 1])  # Сглаживание Y

            #good_new = remove_outliers(good_new)
            
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                motion_vectors.append((a - c, b - d))
                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
        
        img = cv.add(frame, mask)
        
        if motion_vectors:
            avg_vector = np.mean(motion_vectors, axis=0)
            if avg_vector[0] < -1:  # Если движение слева направо
                arrow_start = (250, 900)
                arrow_end = (150, 900)
                cv.arrowedLine(img, arrow_start, arrow_end, (0, 0, 255), 5, tipLength=0.4)
           
            elif avg_vector[0] > 1:  # Если движение справа налево
                arrow_end = (250, 900)
                arrow_start = (150, 900)
                cv.arrowedLine(img, arrow_start, arrow_end, (0, 0, 255), 5, tipLength=0.4)
         


        # Уменьшаем размер кадра перед отображением
        scale_factor = 0.5  # Измени значение, если нужно другое уменьшение (0.5 = уменьшение в 2 раза)
        resized_img = cv.resize(img, (0, 0), fx=scale_factor, fy=scale_factor)
        cv.imshow('frame', resized_img)
        #cv.imshow('frame', img)
        frame_count += 1
        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
        
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    
    return

video_path = "C:/Users/prol-/Downloads/test.mp4"
cap = cv.VideoCapture(video_path, cv.CAP_FFMPEG)
fps = cap.get(cv.CAP_PROP_FPS)
total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
segment_duration = 5

for start_frame in range(int(21 * fps), total_frames, int(segment_duration * fps)):
    process_video_segment(cap, start_frame, segment_duration, fps)
    
cap.release()
cv.destroyAllWindows()


No features detected in the frame 1680. Skipping this segment.


KeyboardInterrupt: 

In [6]:
cv.destroyAllWindows()

In [ ]:
# Python program to explain cv2.rotate() method

# importing cv2
import cv2

# path
path = r'C:\Users\user\Desktop\geeks14.png'

# Reading an image in default mode
src = cv2.imread(path)

# Window name in which image is displayed
window_name = 'Image'

# Using cv2.rotate() method
# Using cv2.ROTATE_90_CLOCKWISE rotate
# by 90 degrees clockwise
image = cv2.rotate(src, cv2.ROTATE_90_CLOCKWISE)

# Displaying the image
cv2.imshow(window_name, image)
cv2.waitKey(0)
